# Parameter Sweep Analysis

Load and analyze results from a FermiHarmonics parameter sweep. This notebook scans a directory of HDF5 files, extracts parameter values from filenames, and provides tools for visualizing and comparing results across the parameter space.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
from pathlib import Path

## Parse filenames to extract parameters

Files are named like `gamma_mc=200.0_gamma_mr=42.9_index_global=1100.h5`

In [ ]:
def parse_filename(filename):
    """
    Parse parameter values from filename.
    
    Args:
        filename: string like 'gamma_mc=0.1_gamma_mr=0.05_index_global=123.h5'
    
    Returns:
        dict with parameter names and values
    """
    # Remove .h5 extension if present
    name = filename[:-3] if filename.endswith('.h5') else filename
    
    # Split by '=' to find key-value pairs
    parts = name.split('_')
    params = {}
    
    i = 0
    while i < len(parts):
        # Look for parts that contain '='
        if '=' in parts[i]:
            key_val = parts[i].split('=')
            if len(key_val) == 2:
                key, val = key_val
                # Try to parse as float
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
            i += 1
        else:
            # This might be part of a multi-word key
            # Look ahead to see if next part has '='
            if i + 1 < len(parts) and '=' in parts[i + 1]:
                # This part is a prefix to the key
                key_val = parts[i + 1].split('=')
                if len(key_val) == 2:
                    key = parts[i] + '_' + key_val[0]
                    val = key_val[1]
                    try:
                        params[key] = float(val)
                    except ValueError:
                        params[key] = val
                    i += 2
                else:
                    i += 1
            else:
                i += 1
    
    return params

# Test the parser
test_name = "gamma_mc=200.0_gamma_mr=42.9_index_global=1100.h5"
print("Example parse:", parse_filename(test_name))

## Scan directory for all runs

Walk through a directory and collect all HDF5 files with their parameter values.

In [ ]:
def scan_runs(root):
    """
    Walk `root` and collect all runs.
    
    Returns:
        list of (params_dict, filepath) tuples
    """
    root = Path(root)
    runs = []
    
    for filename in os.listdir(root):
        if not filename.endswith('.h5'):
            continue
        
        try:
            params = parse_filename(filename)
            filepath = root / filename
            runs.append((params, filepath))
        except Exception as e:
            print(f"Warning: Could not parse {filename}: {e}")
            continue
    
    return runs

# Scan the data directory (adjust path as needed)
root_dir = Path("results/example_sweep/data")
runs = scan_runs(root_dir)

print(f"{len(runs)} runs found")
if runs:
    print(f"First run: {runs[0][0]}")
    print(f"Path: {runs[0][1]}")

## Build indexed parameter grid

Create sorted lists of parameter values and build a mapping from (i, j) indices to file paths.

In [ ]:
def build_parameter_grid(runs, param_x='gamma_mr', param_y='gamma_mc'):
    """
    Build indexed grid for two parameters.
    
    Args:
        runs: list of (params_dict, filepath) tuples
        param_x: name of x-axis parameter
        param_y: name of y-axis parameter
    
    Returns:
        param_x_list: sorted unique values of param_x
        param_y_list: sorted unique values of param_y
        by_ij: dict mapping (i, j) -> (params, filepath)
    """
    # Extract unique sorted parameter values
    x_vals = sorted({params[param_x] for params, _ in runs if param_x in params})
    y_vals = sorted({params[param_y] for params, _ in runs if param_y in params})
    
    # Create index mappings
    x_to_i = {x: i for i, x in enumerate(x_vals)}
    y_to_j = {y: j for j, y in enumerate(y_vals)}
    
    # Build (i, j) -> data mapping
    by_ij = {}
    for params, path in runs:
        if param_x in params and param_y in params:
            i = x_to_i[params[param_x]]
            j = y_to_j[params[param_y]]
            by_ij[(i, j)] = (params, path)
    
    return x_vals, y_vals, by_ij

# Build the grid (adjust parameter names as needed)
gamma_mr_list, gamma_mc_list, by_ij = build_parameter_grid(runs, 'gamma_mr', 'gamma_mc')

print(f"gamma_mr values: {gamma_mr_list}")
print(f"gamma_mc values: {gamma_mc_list}")
print(f"Grid size: {len(gamma_mr_list)} x {len(gamma_mc_list)}")
print(f"Total indexed runs: {len(by_ij)}")

## Load and visualize a single run

Select a run by its (i, j) indices and visualize the observables.

In [ ]:
def load_observables(filepath):
    """
    Load observables from HDF5 file.
    
    Returns:
        dict with keys: x, y, A0, A1, B1, X (meshgrid), Y (meshgrid), metadata
    """
    with h5py.File(filepath, 'r') as f:
        data = {
            'A0': f['a0'][:],
            'A1': f['a1'][:],
            'B1': f['b1'][:],
            'x': f['x'][:],
            'y': f['y'][:],
        }
        
        # Get metadata
        metadata = dict(f.attrs)
        data['metadata'] = metadata
        
    # Create meshgrid for plotting
    data['X'], data['Y'] = np.meshgrid(data['x'], data['y'])
    
    return data

# Example: load a specific run
i, j = 0, 0 # indices in the parameter grid

if (i, j) in by_ij:
    params, filepath = by_ij[(i, j)]
    gamma_mr = params['gamma_mr']
    gamma_mc = params['gamma_mc']
    
    print(f"Loading run at (i={i}, j={j})")
    print(f"  gamma_mr = {gamma_mr}")
    print(f"  gamma_mc = {gamma_mc}")
    print(f"  file = {filepath.name}")
    
    data = load_observables(filepath)
    
    print(f"  Grid shape: {data['A0'].shape}")
    print(f"  Time: {data['metadata'].get('time', 'N/A')}")
else:
    print(f"No run found at (i={i}, j={j})")
    data = None

## Visualization: Density (A0)

Plot the zeroth harmonic (density) as a color map.

In [ ]:
if data is not None:
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Use nan-aware limits so NaNs outside the geometry don't break autoscaling
    vmin = np.nanmin(data['A0'])
    vmax = np.nanmax(data['A0'])

    colormesh = ax.pcolormesh(data['X'], data['Y'], data['A0'], 
                               cmap='viridis', shading='auto', vmin=vmin, vmax=vmax)
    plt.colorbar(colormesh, ax=ax, label='$A_0$ (Density)')
    
    ax.set_xlabel('$x/L$')
    ax.set_ylabel('$y/L$')
    ax.set_title(f'Density $A_0$ ($\\gamma_{{mr}}={gamma_mr:.3g}$, $\\gamma_{{mc}}={gamma_mc:.3g}$)')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

## Visualization: Current (A1)

Plot the x-component of the current.

In [ ]:
if data is not None:
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Use diverging colormap centered at zero; ignore NaNs outside geometry
    vmax = np.nanmax(np.abs(data['A1']))
    colormesh = ax.pcolormesh(data['X'], data['Y'], data['A1'], 
                               cmap='RdBu_r', shading='auto',
                               vmin=-vmax, vmax=vmax)
    plt.colorbar(colormesh, ax=ax, label='$A_1 \\sim j_x$')
    
    ax.set_xlabel('$x/L$')
    ax.set_ylabel('$y/L$')
    ax.set_title(f'Current $A_1$ ($\\gamma_{{mr}}={gamma_mr:.3g}$, $\\gamma_{{mc}}={gamma_mc:.3g}$)')
    ax.set_aspect('equal')
    
    plt.savefig("ballistic_p=1.0.png")
    plt.tight_layout()
    plt.show()

## Visualization: Current streamlines

Show the current field as streamlines with magnitude as color.

In [ ]:
if data is not None:
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Compute current magnitude and use nan-aware color scaling
    current_magnitude = np.sqrt(data['A1']**2 + data['B1']**2)
    mag_vmin = np.nanmin(current_magnitude)
    mag_vmax = np.nanmax(current_magnitude)

    # Create streamplot with explicit normalization so NaNs are ignored
    norm = plt.Normalize(vmin=mag_vmin, vmax=mag_vmax)
    stream = ax.streamplot(data['X'], data['Y'], data['A1'], data['B1'],
                           color=current_magnitude, cmap='coolwarm', norm=norm,
                           density=1.0, linewidth=1.0)

    plt.colorbar(stream.lines, ax=ax, label='$|\\mathbf{j}| = \\sqrt{A_1^2 + B_1^2}$')

    ax.set_xlabel('$x/L$')
    ax.set_ylabel('$y/L$')
    ax.set_title(f'Current Streamlines ($\\gamma_{{mr}}={gamma_mr:.3g}$, $\\gamma_{{mc}}={gamma_mc:.3g}$)')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

## Line cuts: Compare multiple parameters

Extract line cuts at fixed y-position for multiple parameter values.

In [ ]:
# Select which parameter to vary
i_fixed = 0  # Fix gamma_mr index
y_cut = 0.4  # y-position for line cut (adjust based on your domain)

fig, ax = plt.subplots(figsize=(10, 6))

# Loop through gamma_mc values (vary j index)
# Adjust step size to control number of curves plotted
step = max(1, len(gamma_mc_list) // 10)  # Plot ~10 curves evenly spaced
for j in range(0, len(gamma_mc_list), step):
    if (i_fixed, j) not in by_ij:
        continue
    
    params, filepath = by_ij[(i_fixed, j)]
    gamma_mr = params['gamma_mr']
    gamma_mc = params['gamma_mc']
    
    try:
        data_cut = load_observables(filepath)
        
        # Find closest y index
        y_idx = np.argmin(np.abs(data_cut['y'] - y_cut))
        
        # Extract line cut (B1 is y-component of current)
        b1_cut = data_cut['B1'][y_idx, :]
        
        # Compute total integrated y-current along this line cut
        dx = data_cut['x'][1] - data_cut['x'][0] if len(data_cut['x']) > 1 else 1.0
        total_current = np.nansum(b1_cut) * dx
        
        # Normalize by total current so all line cuts have same integral
        if abs(total_current) > 1e-10:
            b1_normalized = b1_cut / total_current
        else:
            b1_normalized = b1_cut
        
        ax.plot(data_cut['x'], b1_normalized, 
                label=f'$\\gamma_{{mc}}={gamma_mc:.3g}$')
    except Exception as e:
        print(f"Error loading (i={i_fixed}, j={j}): {e}")

ax.set_xlabel('$x/L$')
ax.set_ylabel('$B_1 / I_y$ (normalized)')
ax.set_title(f'Normalized line cuts at $y={y_cut}$ ($\\gamma_{{mr}}={gamma_mr_list[i_fixed]:.3g}$)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()